# Creating Custom Data for Microsoft Sentinel - 3rd Party Security Alerts

This notebook will create custom data for Microsoft Sentinel from 3rd party security alerts, in a format that can be ingested by Sentinel.
After running this notebook, you will have sample data in a custom table you've specified, where you'll be able to create analytics rules and workbooks, leveraging Sentinel's capabilities for 3rd party security alerts. 

This is suitable for a POC or demo environment where you want to show how Sentinel can be used to analyze 3rd party security alerts like from a SIEM or other security products.

## Notebook Initialization

In [ ]:
## Import required libraries
import json
import requests
import datetime
import hashlib
import hmac
import base64

In [2]:
######################
###### Functions######
######################

## Build the API signature
def build_signature(customer_id, shared_key, date, content_length, method, content_type, resource):
    x_headers = 'x-ms-date:' + date
    string_to_hash = method + "\n" + \
        str(content_length) + "\n" + content_type + \
        "\n" + x_headers + "\n" + resource
    bytes_to_hash = bytes(string_to_hash, encoding="utf-8")
    decoded_key = base64.b64decode(shared_key)
    encoded_hash = base64.b64encode(hmac.new(
        decoded_key, bytes_to_hash, digestmod=hashlib.sha256).digest()).decode()
    authorization = "SharedKey {}:{}".format(customer_id, encoded_hash)
    return authorization

## Build and send a request to the POST API
def post_data(customer_id, shared_key, body, log_type):
    method = 'POST'
    content_type = 'application/json'
    resource = '/api/logs'
    rfc1123date = datetime.datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')
    content_length = len(body)
    signature = build_signature(
        customer_id, shared_key, rfc1123date, content_length, method, content_type, resource)
    uri = 'https://' + customer_id + '.ods.opinsights.azure.com' + \
        resource + '?api-version=2016-04-01'

    headers = {
        'content-type': content_type,
        'Authorization': signature,
        'Log-Type': log_type,
        'x-ms-date': rfc1123date
    }

    response = requests.post(uri, data=body, headers=headers)
    if (response.status_code >= 200 and response.status_code <= 299):
        print('Accepted')
    else:
        print("Response code: {}".format(response.status_code))

## Write a function to slugfy the name of the log type
def slugify(text):
    text = text.replace(" ", "_")
    return text

## Obtaining required information

In [1]:
## Get workpace id and shared key from Azure Sentinel
customer_id = input("Enter your Workspace ID: ")
shared_key = input("Enter your Workspace Key: ")

log_type = "ELKSecurityAlerts"

if log_type == "":
    log_type = input("Enter the name of the table name: ")

## Slugify the name of the log type
log_type = slugify(log_type)

## Getting the data

In [12]:
## Defines json payload
json_data = [
    {
        "id": "170792011",
        "malfename": "Phish.LIVE.DTI.URL",
        "url_alert": "https: //etp.us.fireeye.com/alert/123456789",
        "mail_from": "office@ccbm-usa.com",
        "domain_from": "ccbm-usa.com",
        "mail_dest": "alexw@davicruz.net",
        "technology": "FireEye ETP",
        "action": "Block",
        "category": "email",
        "hostname": "FireEye",
        "mail_subject": "Contoso Bank Invoice/Statement for Account 3224724 202306 ",
        "file_name": "",
        "file_hash": "",
        "url": "hxxps://wearsomgmg[dot]yachts/?ijvl",
        "rid": "0"
    }]

## Also get content from text file (optional)
# file_path = './data.txt'
# with open(file_path) as json_file:
#     json_data = json.load(json_file)

In [13]:
## Post data to Log Analytics
post_data(customer_id, shared_key, json.dumps(json_data), log_type)

Accepted
